# Capstone Hotel reservation prediction

![Logo](img/hotel-logo.png)

## Modelling

### Import required libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
#import eli5 # https://pypi.org/project/eli5/

#### Loading cleaned dataset

In [3]:
df = pd.read_csv("data/dataset_w_feature.csv", index_col=0, encoding="iso-8859-15")

In [4]:
df["buchungsdatum"]      = pd.to_datetime(df["buchungsdatum"])
df["anreisedatum"]       = pd.to_datetime(df["anreisedatum"])
df["abreisedatum"]       = pd.to_datetime(df["abreisedatum"])

In [5]:
df.head()

,buchungsdatum,anreisedatum,abreisedatum,naechte_insgesamt,gebuchte_naechte,personen_anzahl,gesamtpreis,saison,buchung_telefon,buchung_online,buchung_reisebuero,zimmerarten,reise_adventure,reise_relax,reise_standard,reise_genuss,kind,entfernung,zahler,kunden_id,buchungsnr,buchungsnr_reisebuero,menue_code,zusatzleistung,destination,anrede,plz,ort,kein_email_ad,kein_brief_ad,kein_anruf_ad,kein_katalog_ad,kein_sms_ad,keine_werbung,kundenclub,jahrgangsalter_klassiert,bundesland,einwohner_adjusted_plz,bev_m_plz,bev_w_plz,bev_insgesamt_ort,bev_weiblich_ort,bev_maennlich_ort,qkm_plz,flaeche_ort,einwohnerdichte_prokm2_plz,haushalte_regiog_2017_plz,kaufkraft_einwohner_in_eur_regio,kk_einwohner_index_regiog_2017,kk_hh_index_regiog_2017,gemeindetyp_regiostar_2016,wirtschaftskraftmr1,wirtschaftskraftmr2,wochentag_buchung,buchungstag,buchungsmonat,buchungsjahr,buchungswoche,buchungsquartal,wochentag_anreise,anreisetag,anreisemonat,anreisejahr,anreisewoche,anreisequartal,wochentag_abreise,abreisetag,abreisemonat,abreisejahr,abreisewoche,abreisequartal,buchungskanal,werbeban,aufenthalt,naechte_ausserhalb,buchungs_vorlauf,preis_per,preis_nacht,preis_nacht_per,buch_zeilen,erst_buchung,folgebuchung,repeater
0,2010-01-29,2010-04-02,2010-04-10,8,8,2,2982,2010,0,0,1,classic,0,0,0,1,0,535.48,1,1,1151,456,5,5,linz,Herr,29664,Walsrode,0,0,0,0,0,0,0,55-64,Niedersachsen,25173,12540,12633,25173,11577,11491,344.326464,344.326464,73.108,11453,20641.89,92.819,98.104,Landregion - Zentrale Stadt,-0.333392,-0.388738,Freitag,29,1,2010,4,1,Freitag,2,4,2010,13,2,Samstag,10,4,2010,14,2,reisebuero,keine,8,0,63,1491.00,372.75,186.38,2,1,0,0
1,2010-01-14,2010-04-05,2010-04-10,5,5,2,1437,2010,0,0,1,classic,0,0,1,0,0,544.72,0,2,40,6413,5,5,linz,Frau,50226,Frechen,1,1,1,1,1,1,0,55-64,Nordrhein-Westfalen,52473,25645,26828,52473,26828,25645,45.118960,45.118960,1162.992,26227,25268.58,113.623,111.379,Landregion - Zentrale Stadt,0.853802,-0.345112,Donnerstag,14,1,2010,2,1,Montag,5,4,2010,14,2,Samstag,10,4,2010,14,2,reisebuero,"email, brief, anruf, katalog, sms, generell",5,0,81,718.50,287.40,143.70,2,1,0,0
2,2010-03-01,2010-04-05,2010-04-10,5,5,2,1737,2010,0,0,1,classic,0,0,1,0,0,606.63,0,3,2134,4322,5,5,linz,Herr,23970,Wismar,0,0,0,0,0,0,0,75-84,Mecklenburg-Vorpommern,1423,692,731,46442,21844,20706,50.563027,83.131346,17.117,6355,18697.65,84.076,73.665,"Stadtregion - Kleinstädtischer, dörflicher Raum",-1.012536,-1.012536,Montag,1,3,2010,9,1,Montag,5,4,2010,14,2,Samstag,10,4,2010,14,2,reisebuero,keine,5,0,35,868.50,347.40,173.70,1,1,0,0
3,2010-01-25,2010-03-21,2010-03-25,4,4,2,1047,2010,0,0,1,classic,0,1,0,0,0,421.59,1,4,776,6414,69,9,linz,Frau,15732,Eichwalde,0,0,0,0,0,0,0,65-74,Brandenburg,14690,7302,7388,14690,4135,4087,11.965427,11.965427,1227.704,7159,23442.06,105.410,104.087,Landregion - Städtischer Raum,0.417226,0.417226,Montag,25,1,2010,4,1,Sonntag,21,3,2010,11,1,Donnerstag,25,3,2010,12,1,reisebuero,keine,4,0,55,523.50,261.75,130.88,1,1,0,0
4,2010-01-26,2010-03-07,2010-03-09,2,2,20,5279,2010,0,0,1,classic,1,0,0,0,0,37.88,0,5,813,2938,113,2,duesseldorf,Frau,53881,Euskirchen,0,0,0,0,0,0,1,45-54,Nordrhein-Westfalen,28531,13975,14556,57975,29578,28397,127.679129,139.430913,204.625,11683,23328.82,104.901,120.110,"Stadtregion - Kleinstädtischer, dörflicher Raum",0.212335,-0.579173,Dienstag,26,1,2010,4,1,Sonntag,7,3,2010,9,1,Dienstag,9,3,2010,10,1,reisebuero,keine,2,0,40,263.95,2639.50,131.98,7,1,0,1


# Modelling

#### Dropping redundant variables

In [6]:
df.drop("buchungskanal", axis=1, inplace=True)
df.drop("werbeban", axis=1, inplace=True)

#### Grouping variables

In [7]:
categorial = list(df.columns[df.dtypes==object])
datetime64 = ['buchungsdatum', 'anreisedatum', 'abreisedatum']
numeric    = list(df.select_dtypes(include=['int64', 'float64']).columns)

for entry in ['erst_buchung', 'repeater']: 
    numeric.remove(entry)

df.shape[1] == len(categorial) + len(datetime64) + len(numeric) + 2

True

## Logistic Regression

In [8]:
X = df[numeric]
X.drop([ 'folgebuchung', 'kunden_id'], axis=1)

y = df.folgebuchung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

In [9]:
logistic_regression= LogisticRegression(max_iter=10_000)
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

#### Confusion Matrix & Classification Report

In [10]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[12366,   538],
       [ 2012,  1110]], dtype=int64)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91     12904
           1       0.67      0.36      0.47      3122

    accuracy                           0.84     16026
   macro avg       0.77      0.66      0.69     16026
weighted avg       0.82      0.84      0.82     16026



In [12]:
print('normalized Accuracy: ',round(accuracy_score(y_test, y_pred)*100,2))

normalized Accuracy:  84.09


### Dummy Variable

In [13]:
df_dum = pd.get_dummies(df[categorial])
# bei vielen Ausprägungen bins, erst einmal ohne
df_dum.shape

(160259, 8080)

### Train-Test-Split

In [14]:
X = pd.concat([df[numeric], df_dum], axis=1)
X = X.drop([ 'folgebuchung', 'kunden_id'], axis=1)

y = df.folgebuchung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1337)

### Dealing with imbalace 
in target feature folgebuchung by resampling

In [15]:
df.folgebuchung.value_counts()

0    129462
1     30797
Name: folgebuchung, dtype: int64

In [16]:
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
erstbuchung  = X[X.folgebuchung==0]
folgebuchung = X[X.folgebuchung==1]

# upsample minority
folgebuchung_upsampled = resample(folgebuchung, replace=True, 
                          n_samples=len(erstbuchung), # match number in majority class
                          random_state=1337) 

# combine majority and upsampled minority
upsampled = pd.concat([erstbuchung, folgebuchung_upsampled])

upsampled.folgebuchung.value_counts()

1    116558
0    116558
Name: folgebuchung, dtype: int64

### 2nd Logistic Regression
with balance data and additional dummy variables

In [17]:
y_train = upsampled.folgebuchung
X_train = upsampled.drop('folgebuchung', axis=1)

upsampled = LogisticRegression(solver='liblinear', max_iter=10_000).fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test);

#### Confusion Matrix & Classification Report

In [18]:
n_conf_matrix = confusion_matrix(y_test, upsampled_pred)
n_conf_matrix

array([[11169,  1735],
       [  542,  2580]], dtype=int64)

In [19]:
print(classification_report(y_test, upsampled_pred))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91     12904
           1       0.60      0.83      0.69      3122

    accuracy                           0.86     16026
   macro avg       0.78      0.85      0.80     16026
weighted avg       0.88      0.86      0.87     16026



In [20]:
print('normalized Accuracy: ',round(accuracy_score(y_test, upsampled_pred)*100,2))

normalized Accuracy:  85.79


In [21]:
#https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter

#sklearn.pipeline.Pipeline 

In [22]:
#df_dum.head(2)

In [23]:
#categorial